In [1]:
using JuMP, Gurobi, PyCall
plt = pyimport("matplotlib.pyplot")
;

## File handling

In [2]:
#this function inputs multiple string arrays and outputs a matrix of int and float
function arrayToMatrix(array)
    h=[]
    counter=0
    for element in array
        counter=0
        g=split(element)
        b=[]
        #converting strings to int or float
        for item in g
            counter+=1
            if occursin(".",item)
                push!(b,parse(Float64,item))
                continue
            end
            push!(b,parse(Int64,item))
        end
        push!(h,b)
    end
    #creating an empty matrix and thereafter adding all the arrays in the matrix
    m = Matrix{Any}(nothing, 0, counter)
    for element in h
        m=[m;element']
    end
    return m
end
 
#this function inputs a textfile and outputs a dictionary
function readFromFile(text)
    a=readlines(text)
    dict=Dict([])
    key=""
    array=[]
    flag=false
    for element in a
        if flag==true && occursin(";", element)
            ja=arrayToMatrix(array)
            push!(dict, key=>ja)
            array=[]
            flag=false
            continue
        end
        if flag==true
            push!(array, element)
        end
        if occursin("#",element)
            key=element[2:end]
            flag=true
        end
    end
    return dict
end
;

## Sets

In [3]:
fileName="BP_parameters.txt"

I=readFromFile(fileName)["I"][1]
U=readFromFile(fileName)["U"][1] #
S=readFromFile(fileName)["S"][1]
D=H=readFromFile(fileName)["D"][1]
R=readFromFile(fileName)["R"][1]
M=readFromFile(fileName)["M"][1]
;

## Parameters

In [4]:
Preferanse_ir=readFromFile(fileName)["Preferanse_ir"]
L_isA=readFromFile(fileName)["L_isA"]
T_im=readFromFile(fileName)["T_im"]
G_dr=readFromFile(fileName)["G_dr"]
N_sd=readFromFile(fileName)["N_sd"]
S_sd=readFromFile(fileName)["S_sd"]
D_iuW=readFromFile(fileName)["D_iuW"]
D_iuW_NY=D_iuW/sum(D_iuW)
K_dr=readFromFile(fileName)["K_dr"]
D_iuD=readFromFile(fileName)["D_iuD"]
B_iu=readFromFile(fileName)["B_iu"]
L_sd=readFromFile(fileName)["L_sd"]
A_id=readFromFile(fileName)["A_id"]
P=readFromFile(fileName)["P"][1]
Q_d=readFromFile(fileName)["Q_d"][1]
Q_w=readFromFile(fileName)["Q_w"][1]
Percentage=readFromFile(fileName)["Percentage"][1]
E_iu1=readFromFile(fileName)["E_iu1"]
E_iu2=readFromFile(fileName)["E_iu2"]
E_iu3=readFromFile(fileName)["E_iu3"]
E_iu4=readFromFile(fileName)["E_iu4"]
E_iu5=readFromFile(fileName)["E_iu5"]
E_iud=cat(dims=3,E_iu1,E_iu2,E_iu3, E_iu4, E_iu5)
MDT_interpretation_time_i=readFromFile(fileName)["MDT_interpretation_time_i"]
scanning_time_sd=readFromFile(fileName)["scanning_time_sd"]
;




## Variables

In [5]:
m = Model(with_optimizer(Gurobi.Optimizer))
@variable(m, delta_p[1:D,1:R] >= 0, Int)
@variable(m, delta_m[1:D,1:R] >= 0, Int)
@variable(m, x[1:I,1:U,1:S,1:D] >= 0, Int)
@variable(m, w[1:I,1:U,1:D,1:R,1:M] >= 0, Int)
@variable(m, u_i[1:I,1:U,1:D,1:H,1:R] >= 0, Int)
@variable(m, v[1:I,1:U,1:D,1:H,1:R] >= 0, Int)
@variable(m, alpha[1:I,1:U,1:D], Bin)
@variable(m, beta[1:S,1:D,1:R], Bin)
@variable(m, gamma[1:I,1:D,1:R], Bin)
@variable(m, zeta[1:S,1:D]>=0, Int)#number of open slots at the scanner s at day d
@variable(m, open_task_dr[1:D,1:R] >=0, Int)
;

Academic license - for non-commercial use only


## Objective function

In [6]:
@objective(m, Min, sum(delta_p[d,r]+delta_m[d,r] for d=1:D, r=1:R ));

## Constraints

In [7]:
#Deltas
@constraint(m,[d=1:D, r=1:R], delta_m[d,r]-delta_p[d,r] + open_task_dr[d,r]==G_dr[d,r])             

#minimum number of hours a radiologist can work per day
@constraint(m,[d=1:D,r=1:R], open_task_dr[d,r] + sum(T_im[i,m]*w[i,u,d,r,m]  for i=1:I,u=1:U, m in [1,2])+sum(scanning_time_sd[s,d]*beta[s,d,r] for s=1:S)+sum(MDT_interpretation_time_i[i]*gamma[i,d,r] for i=1:I)>=K_dr[d,r]-Q_d)

#maximum number of hours a radiologist can work per day
@constraint(m,[d=1:D,r=1:R], open_task_dr[d,r] + sum(T_im[i,m]*w[i,u,d,r,m]  for i=1:I,u=1:U, m in [1,2])+sum(scanning_time_sd[s,d]*beta[s,d,r] for s=1:S)+sum(MDT_interpretation_time_i[i]*gamma[i,d,r] for i=1:I)<=K_dr[d,r]+Q_d)

#The total number of working hours for each radiologist must equal Q_w
@constraint(m, [r=1:R], sum(open_task_dr[d,r] for d=1:D)+sum(T_im[i,m]*w[i,u,d,r,m] for i=1:I, u=1:U, d=1:D, m in [1,2]) + sum(scanning_time_sd[s,d]*beta[s,d,r] for s=1:S, d=1:D)+sum(MDT_interpretation_time_i[i]*gamma[i,d,r] for i=1:I, d=1:D)==Q_w)

#The sum of all internal interpretation tasks for u a given day h must equal the number of w[m=1] the same day
#The same goes to v.
@constraint(m,[i=1:I,u=1:U,h=1:H, r=1:R], sum(u_i[i,u,d,h,r] for d=1:D) == sum(w[i,u,h,r,m] for m=1))
@constraint(m,[i=1:I,u=1:U,h=1:H, r=1:R], sum(v[i,u,d,h,r] for d=1:D) == sum(w[i,u,h,r,m] for m=2))

#Making sure that internal interpretations can be done the current and the following week                                                                                          
for i=1:I, u=1:U, d=1:D
    if d+B_iu[i] <= 7
        @constraint(m, [i,u,d], sum(x[i,u,s,d] for s=1:S) - sum(u_i[i,u,d,h,r] for r=1:R,h=d:d+B_iu[i] if h <= D) == 0)
    else
        @constraint(m, [i,u,d], sum(x[i,u,s,d] for s=1:S) - sum(u_i[i,u,d,h,r] for r=1:R,h=d:d+B_iu[i] if h <= D) - sum(u_i[i,u,d,h,r] for r=1:R, h=1:(d+B_iu[i])%7 if h<d) == 0)                                                                                                                      
    end                                                                                                 
end
                                    
#Making sure that external interpretations can be done the current and the following week 
for i=1:I, u=1:U, d=1:D
    if d+B_iu[i,u] <= 7
        @constraint(m, [i,u,d], sum(v[i,u,d,h,r] for r=1:R, h=d:d+B_iu[i,u] if h<=D) == E_iud[i,u,d])
        @constraint(m, [i,u,d], sum(v[i,u,d,h,r] for r=1:R, h=1:d-1) == 0) #Setter starten av uka =0
        @constraint(m, [i,u,d], sum(v[i,u,d,h,r] for r=1:R, h=(d+B_iu[i,u]+1:D) if h <= D)==0)  #20                                                                                                          
    else                                                                                                                  
        @constraint(m, [i,u,d], sum(v[i,u,d,h,r] for r=1:R, h=d:d+B_iu[i,u] if h<=D) + sum(v[i,u,d,h,r] for r=1:R, h=1:(d+B_iu[i,u])%7 if h<d) == E_iud[i,u,d])
        @constraint(m, [i,u,d], sum(v[i,u,d,h,r] for r=1:R,h=((d+B_iu[i,u])%7) + 1:(d-1)) == 0)    #setter midten av uka =0                                                                                                                                    
    end                                                                                                 
end
                                                                                                                                                                                                              
#Connecting D_iuD to alpha
@constraint(m,[i=1:I,u=1:U], sum(alpha[i,u,d] for d=1:D)>=D_iuD[i,u])

#Connecting alpha to x
@constraint(m,[i=1:I,u=1:U, d=1:D], sum(x[i,u,s,d] for s=1:S)>=alpha[i,u,d])

#The number of internal interpretations must equal number of assigned slots
#We have to include this constraint, otherwise the w can be set to meet G_dr
@constraint(m, sum(w[i,u,d,r,m] for i=1:I, u=1:U, d=1:D, r=1:R, m=1)==sum(x[i,u,s,d] for i=1:I, u=1:U, s=1:S, d=1:D))                                                

#Constraints that deals with demand. Either comment out the two first ones, or the third one.
@constraint(m, [i=1:I, u=1:U], sum(x[i,u,s,d] for s=1:S, d=1:D)<=(D_iuW_NY[i,u]+Percentage)*sum(x[i,u,s,d] for i=1:I, u=1:U, s=1:S, d=1:D))
@constraint(m, [i=1:I, u=1:U], sum(x[i,u,s,d] for s=1:S, d=1:D)>=(D_iuW_NY[i,u]-Percentage)*sum(x[i,u,s,d] for i=1:I, u=1:U, s=1:S, d=1:D))
#@constraint(m,[i=1:I,u=1:U], sum(x[i,u,s,d] for s=1:S,d=1:D)>=D_iuW[i,u])
                                                                                                        
#All the available slots has to be filled up                                                                                              
@constraint(m,[s=1:S,d=1:D], sum(x[i,u,s,d] for i=1:I,u=1:U)==N_sd[s,d]-S_sd[s,d])

#Demand for radiologists the MDT-meetings
@constraint(m,[i=1:I,d=1:D], sum(gamma[i,d,r] for r=1:R)==A_id[i,d])

                                                                                      
#Demand for radiologists for monitoring the scanners
@constraint(m,[s=1:S,d=1:D], sum(beta[s,d,r] for r=1:R)==L_sd[s,d])

                                                                                                
#Maximum share of work week that can be devoted to monitoring a scanner
@constraint(m,[r=1:R], sum(scanning_time_sd[s,d]*beta[s,d,r] for s=1:S, d=1:D)<=0.5*Q_w)

#specialized scanners
@constraint(m, [i=1:I,s=1:S], sum(x[i,u,s,d] for u=1:U, d=1:D)<=1000*L_isA[i,s])

                                                                                                        
#EXTENSIONS - NOT INCLUDED IN THE INITIAL MODEL                                                                                                        

#Radiologist specializations
#@constraint(m, [i=1:I, r=1:R], sum(w[i,u,d,r,m] for u=1:U, d=1:D, m=1:M)<=1000*Preferanse_ir[i,r])
;                                                                                                   


## Results

In [8]:
@time begin
    optimize!(m)
    println("Run time: ")
end

println("Status: ",termination_status(m))
println("Primal status: ",primal_status(m))
println("Dual status: ",dual_status(m))
println("Objective value: ", objective_value(m))
arr1=[]

println()
println("****** Variable: x[i,u,s,d] ******")
counter=0
totCount=0
arr=[]
println("Distribution of slots:")
for i=1:I
    for u=1:U
        for s=1:S
            for d=1:D
                counter+=value(x[i,u,s,d])
                totCount+=value(x[i,u,s,d])
            end
        end
        push!(arr,counter)
        counter=0
    end
    push!(arr1,arr)
    arr=[]
end
#println(D_iuW_NY)
#println(arr1/totCount)

println()
println("****** Variable: x[i,u,s,d] ******")
for s=1:S
    println()
    println("Scanner: ",s)
    for d=1:D
        println()
        println("Day: ",d)
        println("__________________")
        for u=1:U
            for i=1:I
            if value(x[i,u,s,d]) != 0
                println("x[",i,",",u,",",s,",",d,"] = ", value(x[i,u,s,d]))
            end
            end
        end
        #println("Open slots = ",value(zeta[s,d]))
    end 
end


println()
println("Radiologist schedules:")
for r=1:R
    println()
    println("Radiologist:",r)
    for d=1:D
        println("Day:", d)
        if value(open_task_dr[d,r])!=0
            println("open_task[",d,",",r,"] = ", value(open_task_dr[d,r]))
        end
        for s=1:S
            if value(beta[s,d,r])!=0
                println("beta[",s,",",d,",",r,"] = ", value(beta[s,d,r]))
            end
        end
            for i=1:I
                if value(gamma[i,d,r])!=0
                        println("gamma[",i,",",d,",",r,"] = ", value(gamma[i,d,r]))
                end
                for u=1:U
                    for m=1:M
                        if value(w[i,u,d,r,m]) != 0
                            println("w[",i,",",u,",",d,",",r,",",m,"] = ", value(w[i,u,d,r,m]))
                        end
                    end
                end
            end
        
    end
end


println()
println("****** Variable: w[i,u,d,r,m] ******")
for d=1:D
    println()
    println("Day: ",d)
    println("__________________")
    for r=1:R
        #println("Patient group: ", i)
        for i=1:I
            for u=1:U
                #println("Radioligst: ",r)
                for m=1:M
                    if value(w[i,u,d,r,m]) != 0
                        println("w[",i,",",u,",",d,",",r,",",m,"] = ", value(w[i,u,d,r,m]))
                    end
                end
            end
        end
    end
end
println()
println("****** Variable: u[i,u,d,h,r] ******")
for i=1:I
    for u=1:U
        for d=1:D
            for h=1:H
                for r=1:R
                    if value(u_i[i,u,d,h,r]) != 0
                        println("u[",i,",",u,",",d,",",h,",",r,"] = ", value(u_i[i,u,d,h,r]))
                    end
                end
            end
        end
    end
end

println()
println("****** Variable: v[i,u,d,h,r] ******")
for i=1:I
    for u=1:U
        for d=1:D
            for h=1:H
                for r=1:R
                    if value(v[i,u,d,h,r]) != 0
                        println("v[",i,",",u,",",d,",",h,",",r,"] = ", value(v[i,u,d,h,r]))
                    end
                end
            end
        end
    end
end

println()
println("****** Variable: delta[d,r] ******")
for d =1:D
    println()
    println("Day: ",d)
    println("__________________")
    for r=1:R
        if value(delta_p[d,r]) != 0
            println("delta_p[",d,",",r,"] = ",value(delta_p[d,r]))
        end
        if value(delta_m[d,r]) != 0
            println("delta_m[",d,",",r,"] = ",value(delta_m[d,r]))
            
        end
    end
end

println()
println("****** Variable: alpha[i,u,d] ******")
for i=1:I
    for u=1:U
        for d=1:D
            if value(alpha[i,u,d]) != 0
                println("alpha[",i,",",u,",",d,"] = ", value(alpha[i,u,d]))
            end
        end
    end
end

println()
println("****** Variable: beta[s,d,r] ******")
for s=1:S
    for d=1:D
        for r=1:R
            if value(beta[s,d,r]) != 0
                println("beta[",s,",",d,",",r,"] = ", value(beta[s,d,r]))
            end
        end
    end
end

println()
println("****** Variable: gamma[i,d,r] ******")
for i=1:I
    for d=1:D
        for r=1:R
            if value(gamma[i,d,r]) != 0
                println("gamma[",i,",",d,",",r,"] = ", value(gamma[i,d,r]))
            end
        end
    end
end

Academic license - for non-commercial use only
Optimize a model with 3472 rows, 20785 columns and 46380 nonzeros
Variable types: 0 continuous, 20785 integer (880 binary)
Coefficient statistics:
  Matrix range     [4e-03, 9e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 1564 rows and 14145 columns
Presolve time: 0.05s
Presolved: 1908 rows, 6640 columns, 23055 nonzeros
Variable types: 0 continuous, 6640 integer (1881 binary)

Root relaxation: objective 3.600000e+01, 2484 iterations, 0.08 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   36.00000    0   39          -   36.00000      -     -    0s
     0     0   36.00000    0   80          -   36.00000      -     -    0s
     0     0   44.00000    0   69          -   44.00000      -     -    0s
     0     0   44.00000    0   26          -   

u[6,2,1,1,2] = 4.0
u[7,1,1,1,6] = 4.0
u[7,1,1,1,8] = 2.0
u[7,2,4,4,7] = 3.0
u[7,2,5,5,10] = 3.0
u[8,1,1,1,13] = 2.0
u[8,1,3,3,1] = 5.0
u[8,1,3,3,11] = 2.0
u[8,1,5,5,12] = 1.0
u[8,2,2,2,4] = 4.0
u[8,2,4,5,11] = 1.0
u[9,1,1,1,4] = 2.0
u[9,2,1,1,11] = 1.0
u[10,1,3,4,4] = 3.0
u[10,2,3,3,3] = 1.0

****** Variable: v[i,u,d,h,r] ******
v[1,1,1,2,8] = 3.0
v[1,1,2,3,13] = 3.0
v[1,1,3,4,1] = 3.0
v[1,1,4,4,11] = 3.0
v[1,1,5,5,1] = 1.0
v[1,1,5,5,8] = 2.0
v[1,2,1,2,3] = 3.0
v[1,2,2,2,11] = 3.0
v[1,2,3,4,8] = 3.0
v[1,2,4,4,3] = 3.0
v[1,2,5,5,13] = 3.0
v[2,1,1,2,6] = 3.0
v[2,1,2,2,9] = 3.0
v[2,1,3,4,3] = 3.0
v[2,1,4,4,1] = 2.0
v[2,1,4,5,13] = 1.0
v[2,1,5,5,8] = 3.0
v[2,2,1,1,4] = 2.0
v[2,2,2,3,8] = 2.0
v[2,2,3,3,7] = 2.0
v[2,2,4,5,6] = 2.0
v[2,2,5,5,1] = 2.0
v[3,1,1,1,12] = 2.0
v[3,1,2,3,5] = 2.0
v[3,1,3,3,2] = 2.0
v[3,1,4,5,11] = 2.0
v[3,1,5,5,4] = 2.0
v[3,2,1,1,13] = 2.0
v[3,2,2,3,13] = 2.0
v[3,2,3,3,5] = 2.0
v[3,2,4,4,10] = 2.0
v[3,2,5,5,4] = 2.0
v[4,1,1,2,5] = 2.0
v[4,1,2,2,2] = 2.0
v[4,1,3,4,11]